In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
def lstsqk(x, y):
    return np.multiply(x, y).mean(axis = 0) / np.multiply(x, x).mean(axis = 0)

In [ ]:
def lstsqkerr(x, y):
    return 1 / np.sqrt(len(x)) * np.sqrt(np.multiply(y, y).mean(axis = 0) / np.multiply(x, x).mean(axis = 0) - lstsqk(x,y) ** 2)

# First

In [ ]:
df1 = pd.read_excel('experimental_data.xlsx', sheet_name = 'Data').reset_index(drop = True)

In [ ]:
tat = 98066.5
fra = tat / 100

In [ ]:
df1['P, Па'] = df1['P, дел'] * fra
df1['ln(U)'] = np.log(df1['U, мкВ'])

In [ ]:
pset = set(df1['P, дел'])
df_list = []
for i in pset:
    df_list.append(df1.loc[df1['P, дел'] == i])

In [ ]:
list(pset)

# General Data

In [ ]:
parr = []
earr = []
for d in df_list:
    p, v = np.polyfit(d['t, с'], d['ln(U)'], deg = 1, cov = True)
    parr.append(p)
    earr.append(np.sqrt(np.diag(v)))

In [ ]:
def calculate_least_squares(df):
    df1 = df[:].copy()
    
    df1['ln(U) * t'] = df1['t, с'] * df1['ln(U)']
    
    df1['t^2, с^2'] = df1['t, с'] * df1['t, с']
    df1['ln(U)^2'] = df1['ln(U)'] * df1['ln(U)']
    
    x_m = df1['t, с'].mean()
    y_m = df1['ln(U)'].mean()
    
    xx_m = df1['t^2, с^2'].mean()
    xy_m = df1['ln(U) * t'].mean()
    yy_m = df1['ln(U)^2'].mean()
    
    return df1, x_m, y_m, xx_m, xy_m, yy_m

In [ ]:
mnkdata = []
for i in range(len(df_list)):
    df, _, _, _, _, _ = calculate_least_squares(df_list[i])
    mnkdata.append(df)

In [ ]:
for i in range(len(mnkdata)):
    mnkdata[i].to_excel('plots/output{}.xlsx'.format(i + 1))

# Plot Linear Scale

In [ ]:
def get_min_max(series, k1 = 3 / 4, k2 = 11 / 10):
    return k1 * min(series), k2 * max(series)

In [ ]:
def plot_data_linear_scale(ax, df, p, e):
    y_min, y_max = get_min_max(df['ln(U)'])
    x_min, x_max = get_min_max(df['t, с'])

    ax.xaxis.set_major_locator(plt.MultipleLocator(100))
    ax.yaxis.set_major_locator(plt.MultipleLocator(0.25))
    ax.xaxis.set_minor_locator(plt.MultipleLocator(20))
    ax.yaxis.set_minor_locator(plt.MultipleLocator(0.05))
    
    plt.ticklabel_format(axis="x", style="sci", scilimits=(0,0))
    plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
    
    ax.set_ylim(bottom = y_min, top = y_max)
    ax.set_xlim(left = x_min, right = x_max)
    
    ax.grid(visible = True, linestyle = '--', which = 'both')
    x = np.linspace(x_min, x_max, 1000)
    ax.scatter(df['t, с'], df['ln(U)'])
    ax.plot(x, p[0] * x + p[1], label = "Линейная модель: $ln(U) = a + b \cdot x$")
    ax.plot([], [], ' ', label = '$b = {k:.3e} \pm {dk:.3e}, a = {C:.3e} \pm {dC:.3e}$'.format(k = p[0], dk = e[0], C = p[1], dC = e[1]))
    
    ax.legend()
    ax.set_xlabel('$t$, с')
    ax.set_ylabel('$ln(U)$')

In [ ]:
for x in range(len(pset)):
    fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(11.7, 8.3))
    fig.suptitle('Рис. {}. Зависимость $ln(U)$ от времени t'.format(x + 1))
    plot_data_linear_scale(ax, df[x], parr[x], earr[x])
    fig.savefig('plots/plot{}.pdf'.format(x + 1))

In [ ]:
pnparr = np.array(parr)
enparr = np.array(earr)

In [ ]:
k = 9.0 # 1/cm
sigma_k = 0.1

v1 = 420 # cm^3
sigma_v1 = 10

v2 = 420 # cm^3
sigma_v2 = 10

In [ ]:
D = k * v1 * v2 / (v1 + v2) * (-pnparr[:, 0])

In [ ]:
sigma_1 = v2 / (v1 * (v1 + v2)) * sigma_v1
sigma_2 = v1 / (v2 * (v1 + v2)) * sigma_v1
sigma_3 = sigma_k / k
sigma_4 = enparr[:, 0] / pnparr[:, 0]

In [ ]:
pnparr

In [ ]:
sigma_D = D * np.sqrt(sigma_1 ** 2 + sigma_2 ** 2 + sigma_3 ** 2 + sigma_4 ** 2)

In [ ]:
df_d = pd.DataFrame()
df_d['P, Па'] = [df['P, Па'].iloc[0] for df in df_list]
df_d['1/P, 1/Па'] = 1 / df_d['P, Па']
df_d['D'] = D
df_d['sigma_D'] = sigma_D

In [ ]:
df_d

# Plot D

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(11.7, 8.3))
fig.suptitle('Рис. {}. Зависимость коэффициента диффузии $D(P)$'.format(7))

y_min, y_max = get_min_max(df_d['D'])
x_min, x_max = get_min_max(df_d['1/P, 1/Па'])

ax.xaxis.set_major_locator(plt.MultipleLocator(0.1e-4))
ax.yaxis.set_major_locator(plt.MultipleLocator(0.5))
ax.xaxis.set_minor_locator(plt.MultipleLocator(0.5e-5))
ax.yaxis.set_minor_locator(plt.MultipleLocator(0.25))

plt.ticklabel_format(axis="x", style="sci", scilimits=(0,0))
plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))

ax.set_ylim(bottom = y_min, top = y_max)
ax.set_xlim(left = x_min, right = x_max)

ax.grid(visible = True, linestyle = '--', which = 'both')
x = np.linspace(x_min, x_max, 1000)
ax.scatter(df_d['1/P, 1/Па'], df_d['D'])
p, v = np.polyfit(df_d['1/P, 1/Па'], df_d['D'], deg = 1, cov = True)
e = np.sqrt(np.diag(v))

ax.plot(x, p[0] * x + p[1], label = "Линейная модель: $D = a + b / P$")
ax.plot([], [], ' ', label = '$b = {k:.3e} \pm {dk:.3e}, a = {C:.3e} \pm {dC:.3e}$'.format(k = p[0], dk = e[0], C = p[1], dC = e[1]))

ax.legend()
ax.set_xlabel('$1 / P$, 1/Па')
ax.set_ylabel('$D$')

fig.savefig('plots/plot{}.pdf'.format(7))

# Use least squares to fit a linear model